# Juan Carlos Perez Ramirez
## Procesamiento de Lenguaje Natural
## Examen 2

### Preprocesamiento de datos

In [2]:
import os
import xml.etree.ElementTree as ET
import re

# Rutas de entrada y salida
split = "train"
dir = "es_" + split
path = "../../corpus/examen2/"
truth = path + dir + '/truth.txt'  # archivo con los identificadores
xml_dir = path + dir  # carpeta donde están los archivos XML
docs_file = path + split + '.csv'

# abrimos los archivos de salida
with open(docs_file, 'w', encoding='utf-8') as f_textos, \
     open(truth, 'r', encoding='utf-8') as f_in:

    for linea in f_in:
        partes = linea.strip().split(":::")
        if len(partes) != 3:
            continue  # Saltamos líneas mal formateadas

        identificador = partes[0]
        genero = partes[1]
        nacionalidad = partes[2]
        archivo_xml = os.path.join(xml_dir, f"{identificador}.xml")

        if not os.path.exists(archivo_xml):
            print(f"Archivo {archivo_xml} no encontrado, saltando.")
            continue

        try:
            tree = ET.parse(archivo_xml)
            root = tree.getroot()
            documentos = root.find('documents')

            # Guardamos todos los tweets en una lista
            tweets = []
            for doc in documentos.findall('document'):
                texto = doc.text.strip()
                texto = re.sub(r"http\S+|www\S+|https\S+", "", texto)  # elimina URLs
                texto = re.sub(r"@\w+", "<user>", texto) # sustituye menciones por "<user>"
                texto = texto.replace('\n', ' ')  # elimina saltos de línea dentro del tweet
                texto = texto.lower()  # pasar a minúsculas
                tweets.append(texto)

            if tweets:
                linea_usuario = " </s> ".join(tweets) + " </s> " + genero + " </s> " + nacionalidad + "\n"
                f_textos.write(linea_usuario)

        except Exception as e:
            print(f"Error procesando {archivo_xml}: {e}")

In [1]:
from transformers import BertTokenizer
import pandas as pd
import torch
from torch.utils.data import Dataset

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class AuthorProfilingDataset(Dataset):
    def __init__(self, path_txt, max_len=1000, tokenizer=tokenizer):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.load_data(path_txt)
    

    def load_data(self, path_txt):
        texts = []
        genders = []
        nationalities = []

        with open(path_txt, 'r', encoding='utf-8') as f:
            for line in f:
                if "</s>" in line:
                    partes = line.strip().split("</s>")
                    if len(partes) < 3:
                        continue
                    *tweets, genero, nacionalidad = partes
                    texto_completo = " </s> ".join(tweets)  # concatena los tweets
                    texts.append(texto_completo)
                    genders.append(genero)
                    nationalities.append(nacionalidad)


        self.data = pd.DataFrame({'text': texts, 'gender': genders, 'nationality': nationalities})
        self.data['gender'] = self.data['gender'].str.lower().str.strip()
        self.data['nationality'] = self.data['nationality'].str.lower().str.strip()

        self.gender_map = {'male': 0, 'female': 1}
        self.nat_map = {l: i for i, l in enumerate(sorted(self.data['nationality'].unique()))}
        self.data['gender'] = self.data['gender'].map(self.gender_map)
        self.data['nationality'] = self.data['nationality'].map(self.nat_map)

        print(f"Nationalities: {self.nat_map}")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data.iloc[idx]['text']
        gender = self.data.iloc[idx]['gender']
        nationality = self.data.iloc[idx]['nationality']

        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['gender'] = torch.tensor(gender, dtype=torch.long)
        item['nationality'] = nationality
        return item

/home/juancho/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
from torch import nn
from transformers import BertModel

class BERTAuthorClassifier(nn.Module):
    def __init__(self, n_generos, n_nacionalidades):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        hidden_size = self.bert.config.hidden_size
        self.dropout = nn.Dropout(0.3)
        self.fc_genero = nn.Linear(hidden_size, n_generos)
        self.fc_nacionalidad = nn.Linear(hidden_size, n_nacionalidades)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [CLS] embedding
        pooled_output = self.dropout(pooled_output)
        genero_logits = self.fc_genero(pooled_output)
        nacionalidad_logits = self.fc_nacionalidad(pooled_output)
        return genero_logits, nacionalidad_logits


In [3]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch.nn.functional as F

def train(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        genero = batch["gender"].to(device)
        nacionalidad = batch["nationality"].to(device)

        optimizer.zero_grad()
        out_gen, out_nat = model(input_ids, attention_mask)

        loss_gen = F.cross_entropy(out_gen, genero)
        loss_nat = F.cross_entropy(out_nat, nacionalidad)
        loss = loss_gen + loss_nat
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)


In [6]:
train_dataset = AuthorProfilingDataset("../../corpus/examen2/train.csv", max_len=512, tokenizer=tokenizer)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTAuthorClassifier(n_generos=2, n_nacionalidades=len(train_dataset.nat_map)).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 3
for epoch in range(num_epochs):
    loss = train(model, train_loader, optimizer, device)
    print(f"Época {epoch+1}: pérdida = {loss:.4f}")

Nationalities: {'argentina': 0, 'chile': 1, 'colombia': 2, 'mexico': 3, 'peru': 4, 'spain': 5, 'venezuela': 6}


/home/juancho/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


KeyboardInterrupt: 

In [9]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.0.1+cu117
True
